In [1]:
%%capture
%load_ext autoreload
%autoreload 2

# Windows Environment Simulation
from dotenv import load_dotenv
load_dotenv()

### (ExPred) Load the pretrained model

In [2]:
%%capture
from src.expred import (seeding, ExpredInput,
                        BertTokenizerWithSpans, ExpredConfig, Expred)
from src.expred.models import (prepare_for_cl, train_evidence_classifier,
                               train_mtl_token_identifier)
from transformers import BertTokenizer

# or, simply
expred_config = ExpredConfig(
    pretrained_dataset_name='fever',
    base_dataset_name='fever',
    device='cpu',
    load_from_pretrained=True)

# seeding
seeding(1234)

# Initialize tokenizer
tokenizer = BertTokenizerWithSpans.from_pretrained('bert-base-uncased')

# create the model
expred = Expred.from_pretrained(expred_config)
expred.eval()


## Evaluation of subsequences
The black-box model of ExPred that we test for short-cuts, can be applied to automate fact-checking.
The dataset curated for that purpose is called FeVer: a dataset for Fact Verification.

In [8]:
# Read subset of FeVer corresponding to all occurences of a subsequence
import pandas as pd

evaluations = pd.read_json("does_not_have.json")

print("Loaded", len(evaluations) , "items")

Loaded 190 items


In [9]:
# Create a function to simplify execution of ExPred
def evaluate_query_with_expred(current_query, evidence):
    # transform the input to the way the expred accepts
    expred_input = ExpredInput(
        queries=[current_query.split()],
        docs=[evidence.split()],
        labels=['SUPPORTS', 'REFUTES'],
        config=expred_config,
        ann_ids=['spontan_1'],
        span_tokenizer=tokenizer)
    # don't forget to preprocess
    expred_input.preprocess()

    # the output is in the form of a dict:
    expred_output = expred(expred_input)

    # retrieve the evaluation label
    current_output = expred_input.get_decoded_cls_preds(expred_output)
    return [current_query, current_output[0]]

### Subsequence: 'does not have'

First we will confirm that the subsequence holds true for all occurrences of the subsequence.
From the observations it can be seen that all cases of the subset containing the regex are refuted.

In [11]:
# Batch size
n = len(evaluations)

results = []

for i in range (0, n):
    if(evaluations['query'][i]):
        current_query = evaluations['query'][i]
        current_doc = evaluations['evidences'][i]

        results.append(evaluate_query_with_expred(current_query, current_doc))

df = pd.DataFrame(results, columns = ['query', 'label'])
df


,query,label
0,Ireland does not have relatively low-lying mou...,REFUTES
1,Aruba does not have cacti.,REFUTES
2,The Sopranos does not have more than one season.,REFUTES
3,Ireland does not have a central plain.,REFUTES
4,Moscow does not have museums.,REFUTES
...,...,...
185,Islam does not have any followers.,REFUTES
186,Islam does not have any followers.,REFUTES
187,Islam does not have any followers.,REFUTES
188,Islam does not have any followers.,REFUTES


Percentage of confirmed cases:

In [12]:
(((df.label.values == 'REFUTES').sum())/n)*100

100.0

## Adverserial Attack
We replace the substring with another substring that guarantees that the meaning has flipped.

In [18]:
substring = "does not have"
replacewith = "has"
n = len(evaluations)

results = []

for i in range (0, n):
    current_query = evaluations['query'][i].replace(substring, replacewith)
    current_doc = evaluations['evidences'][i]

    results.append(evaluate_query_with_expred(current_query, current_doc))

df = pd.DataFrame(results, columns = ['query', 'label'])
df

,query,label
0,Ireland has relatively low-lying mountains.,REFUTES
1,Aruba has cacti.,SUPPORTS
2,The Sopranos has more than one season.,SUPPORTS
3,Ireland has a central plain.,SUPPORTS
4,Moscow has museums.,SUPPORTS
...,...,...
185,Islam has any followers.,SUPPORTS
186,Islam has any followers.,SUPPORTS
187,Islam has any followers.,SUPPORTS
188,Islam has any followers.,SUPPORTS


## Conclusion

This experiment means that 'has only ever been' is a <span style="color:green">valid short-cut for the ExPred model</span>, as it has very high probability that ExPred refutes a query containing this subsequence. The probability is acquired as follows:

In [19]:
(((df.label.values == 'SUPPORTS').sum())/n)*100

83.6842105263158